In [1]:
import sys
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')
from utils import io
from utils import tdmstudio

import re
import pandas as pd
################
# INPUT FILES  #
################
labeled_datafile = '/home/ec2-user/SageMaker/mariano/datasets/multiculturalism/files/labeled_data.csv'
unlabeled_datafile = '/home/ec2-user/SageMaker/serperi/system/sessions/scal/One_second_round/data/exported_data_2022-11-23_00-18.csv'

################
# OUTPUT FILES #
################
new_ranking_file = '/home/ec2-user/SageMaker/mariano/datasets/multiculturalism/files/new_scores.csv'

######################################
# READING LABELED AND UNLABELED DATA #
######################################


labeled_df = pd.read_csv(labeled_datafile)
labeled_df['id']=[str(id_) for id_ in labeled_df['id']]
assert all([label=='R' or label=='I' for label in labeled_df['label']])
unlabeled_df = pd.read_csv(unlabeled_datafile)
unlabeled_df['id'] = [re.sub('https://proquest.com/docview/','' ,url ) for url in unlabeled_df['URL']]
io.info(f'labeled_df.shape=       {labeled_df.shape}')
io.info(f'unlabeled_df.shape=     {unlabeled_df.shape}')
print()

# FILTERING, ONLY KEEPING SUGGESTIONS, NOT LABELED #
io.info(f'Removing labeled from suggestions (unlabeled) ...')
unlabeled_df=unlabeled_df[unlabeled_df['relevant_or_suggested']=='sugg']
io.info(f'new unlabeled_df.shape= {unlabeled_df.shape}')

# REMOVING LABELED DURING EVALUATION #
io.info(f'Removing labeled from unlabeled ...')
labeled_ids=set(labeled_df['id'])
unlabeled_df = unlabeled_df[[id_ not in labeled_ids for id_ in unlabeled_df['id']]]
io.info(f'new unlabeled_df.shape= {unlabeled_df.shape}')

########################################
# READING TITLE AND TEXT FOR ALL ITEMS #
########################################
print()
io.info('Appending texts ...')
labeled_df['text']=[tdmstudio.get_title_and_text(tdmstudio.get_filename(id_)) for id_ in labeled_df['id']]
unlabeled_df['text']=[tdmstudio.get_title_and_text(tdmstudio.get_filename(id_)) for id_ in unlabeled_df['id']]
io.ok('Done')


2022-12-07 22:15:08.430406 [ INFO  ] labeled_df.shape=       (737, 2)
2022-12-07 22:15:08.430570 [ INFO  ] unlabeled_df.shape=     (8176, 4)

2022-12-07 22:15:08.430715 [ INFO  ] Removing labeled from suggestions (unlabeled) ...
2022-12-07 22:15:08.433632 [ INFO  ] new unlabeled_df.shape= (8115, 4)
2022-12-07 22:15:08.433746 [ INFO  ] Removing labeled from unlabeled ...
2022-12-07 22:15:08.436878 [ INFO  ] new unlabeled_df.shape= (8008, 4)

2022-12-07 22:15:08.437060 [ INFO  ] Appending texts ...
2022-12-07 22:15:18.121014 [  OK   ] Done


In [2]:
##################
# CREATING MODEL #
##################
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from utils import nlp_auxiliary
import spacy

nlp = spacy.load('en_core_web_sm')
stopwords = {stopword for stopword in nlp.Defaults.stop_words if stopword==nlp_auxiliary.preprocessor(stopword)}
vectorizer = TfidfVectorizer(lowercase=True,
                             preprocessor=nlp_auxiliary.preprocessor,
                             stop_words=stopwords,
                             ngram_range=(1,3),
                             max_features=10000,
                             use_idf=True,                             
                             smooth_idf=True,
                             
                            )

io.info('Creating labeled articles representation ...')
X = vectorizer.fit_transform(labeled_df['text'])
y = np.array([1 if label=='R' else 0 for label in labeled_df['label']])

io.info('Training model ...')
model = LogisticRegression()
model.fit(X,y)


io.info('Making predictions ...')
yhat = model.predict_proba(vectorizer.fit_transform(unlabeled_df['text']))
unlabeled_df['new_score']=yhat[:,1]

##################################
# Adding title and removing text #
##################################
unlabeled_df['title'] = [tdmstudio.get_title(tdmstudio.get_filename(id_)) for id_ in unlabeled_df['id']]
unlabeled_df= unlabeled_df.drop(columns=['text'])
unlabeled_df.head()
unlabeled_df.to_csv(new_ranking_file)
io.ok('Done')

2022-12-07 22:15:23.316394 [ INFO  ] Creating labeled articles representation ...
2022-12-07 22:16:10.468838 [ INFO  ] Training model ...
2022-12-07 22:16:10.567868 [ INFO  ] Making predictions ...


ValueError: Wrong number of items passed 2, placement implies 1